In [ ]:
# Ejercicio 13: Análisis de Productos Complementarios

print("=== Análisis de Productos Complementarios ===\n")

# 1. Identificar órdenes con más de una familia de productos
familias_por_orden = data.groupby('order_id', as_index=False)['product_family'].nunique()
familias_por_orden.columns = ['order_id', 'num_familias']

# Filtrar órdenes con 2 o más familias
ordenes_multifamilia = familias_por_orden[familias_por_orden['num_familias'] > 1]['order_id']
print(f"Total de órdenes: {len(familias_por_orden)}")
print(f"Órdenes con múltiples familias: {len(ordenes_multifamilia)} ({len(ordenes_multifamilia)/len(familias_por_orden)*100:.2f}%)\n")

# Filtrar datos para análisis
datos_multifamilia = data[data['order_id'].isin(ordenes_multifamilia)]

# 2. Crear combinaciones de familias por orden
from itertools import combinations

combinaciones_lista = []

for order_id in ordenes_multifamilia:
    # Obtener familias únicas de la orden
    familias = datos_multifamilia[datos_multifamilia['order_id'] == order_id]['product_family'].unique()
    
    # Crear todas las combinaciones de pares
    if len(familias) >= 2:
        for combo in combinations(sorted(familias), 2):
            # Calcular ingresos de esa orden
            ingresos_orden = datos_multifamilia[datos_multifamilia['order_id'] == order_id]['price'].sum()
            combinaciones_lista.append({
                'familia_1': combo[0],
                'familia_2': combo[1],
                'order_id': order_id,
                'ingresos_orden': ingresos_orden
            })

# Convertir a DataFrame
combinaciones_df = pd.DataFrame(combinaciones_lista)

# Analizar frecuencia de combinaciones
analisis_combinaciones = combinaciones_df.groupby(['familia_1', 'familia_2'], as_index=False).agg({
    'order_id': 'nunique',  # Veces que se compran juntas
    'ingresos_orden': 'sum'  # Ingresos totales
})

analisis_combinaciones.columns = ['familia_1', 'familia_2', 'veces_juntas', 'ingresos_totales']

# Calcular porcentajes
# ¿Cuándo se compra familia_1, qué % de veces se compra con familia_2?
total_ordenes_por_familia = data.groupby('product_family')['order_id'].nunique().to_dict()

analisis_combinaciones['porcentaje_familia_1'] = (
    analisis_combinaciones.apply(
        lambda x: (x['veces_juntas'] / total_ordenes_por_familia[x['familia_1']]) * 100,
        axis=1
    )
).round(2)

analisis_combinaciones['porcentaje_familia_2'] = (
    analisis_combinaciones.apply(
        lambda x: (x['veces_juntas'] / total_ordenes_por_familia[x['familia_2']]) * 100,
        axis=1
    )
).round(2)

# Calcular ticket medio de la combinación
analisis_combinaciones['ticket_medio'] = (
    analisis_combinaciones['ingresos_totales'] / analisis_combinaciones['veces_juntas']
).round(2)

# Ordenar por frecuencia
analisis_combinaciones_freq = analisis_combinaciones.sort_values('veces_juntas', ascending=False)

print("=" * 100)
print("TOP COMBINACIONES MÁS FRECUENTES")
print("=" * 100)
print(analisis_combinaciones_freq.head(10).to_string(index=False))
print()

# Ordenar por ingresos
analisis_combinaciones_rent = analisis_combinaciones.sort_values('ingresos_totales', ascending=False)

print("=" * 100)
print("TOP COMBINACIONES MÁS RENTABLES")
print("=" * 100)
print(analisis_combinaciones_rent.head(10).to_string(index=False))
print()

# 3. RECOMENDACIONES
print("=" * 100)
print("RECOMENDACIONES DE CROSS-SELLING")
print("=" * 100)

# Generar recomendaciones por familia
familias_unicas = pd.concat([
    analisis_combinaciones['familia_1'], 
    analisis_combinaciones['familia_2']
]).unique()

for familia in sorted(familias_unicas):
    # Buscar las mejores combinaciones con esta familia
    combos_familia = analisis_combinaciones[
        (analisis_combinaciones['familia_1'] == familia) | 
        (analisis_combinaciones['familia_2'] == familia)
    ].copy()
    
    # Obtener el complemento correcto
    combos_familia['complemento'] = combos_familia.apply(
        lambda x: x['familia_2'] if x['familia_1'] == familia else x['familia_1'],
        axis=1
    )
    
    combos_familia['porcentaje_relevante'] = combos_familia.apply(
        lambda x: x['porcentaje_familia_1'] if x['familia_1'] == familia else x['porcentaje_familia_2'],
        axis=1
    )
    
    # Ordenar por frecuencia
    combos_familia = combos_familia.sort_values('veces_juntas', ascending=False)
    
    if len(combos_familia) > 0:
        mejor_combo = combos_familia.iloc[0]
        print(f"\n📦 {familia.upper()}")
        print(f"   → Combinar con: {mejor_combo['complemento']}")
        print(f"   → Se compran juntos {mejor_combo['veces_juntas']} veces ({mejor_combo['porcentaje_relevante']:.1f}% de las compras de {familia})")
        print(f"   → Ingresos generados: €{mejor_combo['ingresos_totales']:.2f}")
        print(f"   → Ticket medio de la combinación: €{mejor_combo['ticket_medio']:.2f}")

# 4. INSIGHTS PARA ESTRATEGIAS DE MARKETING
print(f"\n{'=' * 100}")
print("INSIGHTS Y ESTRATEGIAS RECOMENDADAS")
print("=" * 100)

combo_top_freq = analisis_combinaciones_freq.iloc[0]
combo_top_rent = analisis_combinaciones_rent.iloc[0]

print(f"\n💡 INSIGHT 1: Combinación más popular")
print(f"   {combo_top_freq['familia_1']} + {combo_top_freq['familia_2']}")
print(f"   • Se compran juntos en {combo_top_freq['veces_juntas']} órdenes")
print(f"   • Representa el {(combo_top_freq['veces_juntas']/len(ordenes_multifamilia)*100):.1f}% de las órdenes multifamilia")
print(f"   ✅ Recomendación: Crear pack promocional o descuento por compra conjunta")

print(f"\n💰 INSIGHT 2: Combinación más rentable")
print(f"   {combo_top_rent['familia_1']} + {combo_top_rent['familia_2']}")
print(f"   • Ingresos totales: €{combo_top_rent['ingresos_totales']:.2f}")
print(f"   • Ticket medio: €{combo_top_rent['ticket_medio']:.2f}")
print(f"   ✅ Recomendación: Destacar esta combinación en campañas premium")

# Calcular el % promedio de cross-selling
avg_cross_selling = analisis_combinaciones['veces_juntas'].mean()
print(f"\n📊 INSIGHT 3: Oportunidad de cross-selling")
print(f"   • Promedio de co-compra: {avg_cross_selling:.1f} veces por combinación")
print(f"   • Solo el {len(ordenes_multifamilia)/len(familias_por_orden)*100:.1f}% de órdenes tienen múltiples familias")
print(f"   ✅ Recomendación: Implementar sugerencias de productos complementarios en el checkout")

# Identificar la familia más "social" (se compra más con otras)
familia_social = combinaciones_df.groupby(['familia_1', 'familia_2']).size().reset_index(name='count')
familia_counts = pd.concat([
    familia_social['familia_1'],
    familia_social['familia_2']
]).value_counts()

print(f"\n🤝 INSIGHT 4: Familia más complementaria")
print(f"   • {familia_counts.index[0]} aparece en más combinaciones diferentes")
print(f"   ✅ Recomendación: Usar {familia_counts.index[0]} como 'producto ancla' para promociones cruzadas")

print(f"\n{'=' * 100}")

# Ejercicios Avanzados de Pandas

Este notebook contiene ejercicios elaborados para practicar operaciones complejas con pandas. Cada ejercicio requiere combinar múltiples técnicas y pensamiento analítico.

**Instrucciones:**
- Lee cada enunciado cuidadosamente
- Los ejercicios requieren combinar varias operaciones de pandas
- Intenta resolver cada problema paso a paso
- Algunos ejercicios tienen múltiples soluciones válidas

In [3]:
!pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 11.0 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 6.7 MB/s  0:00:02m0:00:0100:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]


In [4]:
# Importar librerías
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [5]:
# Cargar los datos
data = pd.read_csv("material/data.csv")
data.head(10)

,order_id,fecha,order_customer_id,product_id,unit_price,quantity,price,product_family,latitude,longitude
0,1016330781887,2023-05-29,6451824076,IAM925P00XXZPUDIA,19.0,8,152.0,Pulsera,37.788617,-3.790215
1,1095376438463,2023-07-16,664062345899,OTS925P00XXZPUDOT,19.0,6,114.0,Pulsera,40.299542,-3.926774
2,1047641588927,2023-06-17,642133246635,UND925P00XXZCOMUN,25.0,5,125.0,Collar,39.651927,-0.411277
3,1181322118257,2023-08-29,714672128644,OTS925A00XXZPUDOT,19.0,4,76.0,Pulsera,39.456511,-0.346203
4,1017492866239,2023-05-30,196440539724,IMO925P00XXZTOMIM,25.0,3,75.0,Tobillera,37.394171,-5.957857
5,480728843265,2023-03-27,226613674572,UDL925P00XXVPUSUD,25.0,3,75.0,Pulsera,38.984944,-3.927849
6,457358836737,2023-03-10,244904182348,UND925P00XXZCOMUN,25.0,3,75.0,Collar,40.604510,-4.339488
7,1001207956671,2023-05-21,621451002539,OTS925P00XXZTODOT,19.0,3,57.0,Tobillera,40.804126,0.516218
8,1057963181247,2023-06-23,647359906475,UND925P00XXZCOMUN,25.0,3,75.0,Collar,41.361692,2.129037
9,1095376438463,2023-07-16,664062345899,OTS925P00XXZPUDOT,19.0,3,57.0,Pulsera,40.299542,-3.926774


---

### Ejercicio 1: Clientes VIP
Identifica a los clientes VIP basándote en los siguientes criterios:
- Han realizado al menos 3 órdenes diferentes
- Su gasto total supera los 100€

Crea un DataFrame con:
- `order_customer_id`
- Número de órdenes
- Gasto total
- Una columna booleana `es_vip` que indique si cumple los 2 criterios

Ordena el resultado por gasto total descendente.

In [4]:
# SOLUCION

# Agrupar por cliente y calcular métricas
clientes = data.groupby('order_customer_id', as_index = False).agg({
    'order_id': 'nunique',  # Número de órdenes diferentes
    'price': 'sum' # Gasto total 
})

clientes.columns = ['order_customer_id', 'num_ordenes', 'gasto_total']

# Crear columna es_vip con los 2 criterios
clientes['es_vip'] = (
    (clientes['num_ordenes'] >= 3) & 
    (clientes['gasto_total'] > 100) 
)

# Ordenar por gasto total descendente
clientes_vip = clientes.sort_values('gasto_total', ascending=False)

print(f"Total de clientes VIP: {clientes_vip['es_vip'].sum()}")
print(f"\nPrimeros 10 clientes:")
clientes_vip.head(10)

Total de clientes VIP: 53

Primeros 10 clientes:


,order_customer_id,num_ordenes,gasto_total,es_vip
93,5753058444,4,432.70,True
498,254713512524,6,431.00,True
419,243532710476,5,425.25,True
895,626415125163,1,346.00,False
38,4594553932,8,346.00,True
218,7012581324,5,326.80,True
635,600548623019,4,308.75,True
878,624689627819,5,259.25,True
1129,663846044331,4,258.15,True
1134,664062345899,2,247.00,False


### Ejercicio 2: Análisis de Cestas de Compra
Para cada orden (`order_id`), calcula:
- Número total de productos en la orden
- Número de unidades totales compradas
- Importe total de la orden
- Número de familias de productos diferentes en la orden
- gasto medio por order

Luego identifica:
- Las 5 órdenes con mayor diversidad de productos (más familias diferentes)
- Las 5 órdenes con mayor importe total
- ¿Hay solapamiento entre estas dos listas?

In [5]:
# Ejercicio 2: Análisis de Cestas de Compra

# Calcular métricas por orden
compras = data.groupby('order_id', as_index= False).agg({
    'product_id': 'nunique',  # Número de productos
    'quantity': 'sum',  # Unidades totales
    'price': ['sum', "mean"],  # Importe total y gasto medio
    'product_family': 'nunique',  # Familias diferentes
})

compras.columns = ['order_id', 'num_productos', 'unidades_totales', 'importe_total', 'gasto_medio', 'num_familias' ]

print("=== Análisis de Cestas de Compra ===\n")

# Top 5 órdenes con mayor diversidad
print("Top 5 órdenes con mayor diversidad de productos:")
top_diversidad = compras.sort_values('num_familias', ascending=False).head(5)
print(top_diversidad)
print("--------")
# Top 5 órdenes con mayor importe
print("Top 5 órdenes con mayor importe total:")
top_importe = compras.sort_values('importe_total', ascending=False).head(5)
print(top_importe)
print("--------")
# Verificar solapamiento
ordenes_diversidad = set(top_diversidad['order_id'])
ordenes_importe = set(top_importe['order_id'])
solapamiento = ordenes_diversidad.intersection(ordenes_importe)
print(f"Órdenes que aparecen en ambas listas: {len(solapamiento)}")
if solapamiento:
    print(f"IDs: {solapamiento}")
else:
    print("No hay solapamiento entre ambas listas")

=== Análisis de Cestas de Compra ===

Top 5 órdenes con mayor diversidad de productos:
           order_id  num_productos  unidades_totales  importe_total  \
1132  1069883917503              9                 9         187.25   
1169  1076911670463              6                 6         135.00   
1038  1047824303295              4                 4          88.00   
380    933853136063              4                 4         104.00   
194    479898305537              4                 4         114.00   

      gasto_medio  num_familias  
1132    20.805556             5  
1169    22.500000             5  
1038    22.000000             4  
380     26.000000             4  
194     28.500000             4  
--------
Top 5 órdenes con mayor importe total:
           order_id  num_productos  unidades_totales  importe_total  \
788   1012895974591              2                 4         346.00   
669    992577586367              1                 1         199.00   
1132  1069883917503  

### Ejercicio 3: Patrón de Compra Recurrente
Identifica qué clientes compran regularmente (tienen órdenes en al menos 3 fechas diferentes) y cuál es su familia de productos favorita (la que más compran en términos de cantidad total).

Crea un DataFrame que muestre:
- `order_customer_id`
- Número de días diferentes en que ha comprado
- Familia de productos favorita
- Cantidad total comprada de esa familia
- Porcentaje que representa esa familia sobre sus compras totales

In [ ]:
# Obtenemos numero de fechas distintas y cantidad total por cliente
fechas_y_cantidad_total = data.groupby("order_customer_id", as_index = False).agg(
    {
        "fecha": "nunique", 
        "quantity": "sum"
    }
    )
fechas_y_cantidad_total.columns = ["order_customer_id", "num_fechas", "cantidad_total"]

# Obtenemos la familia de productos favorita por cliente
compras_por_familia = data.groupby(["order_customer_id", "product_family"], as_index = False)["quantity"].sum()
familia_favorita = compras_por_familia.loc[compras_por_familia.groupby("order_customer_id")["quantity"].idxmax()]
familia_favorita.columns = ["order_customer_id", "familia_favorita", "cantidad_familia_favorita"]

# Unimos ambos dataframes y calculamos el porcentaje de la familia favorita
resultado = fechas_y_cantidad_total.merge(familia_favorita, on = "order_customer_id")
resultado["porcentaje_familia_favorita"] = 100 * resultado["cantidad_familia_favorita"] / resultado["cantidad_total"]
resultado.drop(columns = ["cantidad_total"], inplace = True)

print("=== Análisis de Clientes ===\n")
print("Primeros 10 clientes analizados:")
print(resultado.head(10))

# Clientes que han comprado en más de 2 fechas distintas
print(f"\nNúmero de clientes que han comprado en más de 2 fechas distintas: ")
print(resultado[resultado["num_fechas"] > 2])


=== Análisis de Clientes ===

Primeros 10 clientes analizados:
   order_customer_id  num_fechas familia_favorita  cantidad_familia_favorita  \
0         4438242304           3           Collar                          2   
1         4438303040           1           Collar                          2   
2         4438358016           1        Tobillera                          1   
3         4438388032           1           Anillo                          1   
4         4438411456           1           Collar                          3   
5         4438414656           1          Pulsera                          2   
6         4438568448           1       Pendientes                          3   
7         4438581888           2           Collar                          2   
8         4438603712           1           Collar                          1   
9         4438679360           1           Collar                          3   

   porcentaje_familia_favorita  
0                    40

### Ejercicio 4: Tendencia de Ventas
Convierte la columna `fecha` a datetime y realiza el siguiente análisis:

1. Calcula las ventas diarias totales (suma de `price`)
2. Añade una columna con la diferencia porcentual respecto al día anterior
3. Identifica los 3 días con mayor crecimiento porcentual
4. Identifica los 3 días con mayor caída porcentual


In [7]:
# Ejercicio 4: Tendencia de Ventas

# Convertir fecha a datetime (si no lo está ya)
data['fecha'] = pd.to_datetime(data['fecha'])

# Calcular ventas diarias
ventas_diarias = data.groupby('fecha', as_index = False)['price'].sum()
ventas_diarias.columns = ['fecha', 'ventas_totales']
ventas_diarias = ventas_diarias.sort_values('fecha')

# Calcular diferencia porcentual respecto al día anterior
ventas_diarias['ventas_dia_anterior'] = ventas_diarias['ventas_totales'].shift(1)

ventas_diarias['diferencia_porcentual'] = (
    100 * (ventas_diarias['ventas_totales'] - ventas_diarias['ventas_dia_anterior']) / ventas_diarias['ventas_dia_anterior']
).round(2)
ventas_diarias

# Top 3 días con mayor crecimiento
print("Top 3 días con MAYOR crecimiento porcentual:")
top_crecimiento = ventas_diarias.sort_values('diferencia_porcentual', ascending=False).head(3)
print(top_crecimiento)
print("--------")

# Top 3 días con mayor caída
print("Top 3 días con MAYOR caída porcentual:")
top_caida = ventas_diarias.sort_values('diferencia_porcentual', ascending=True).head(3)
print(top_caida)


Top 3 días con MAYOR crecimiento porcentual:
         fecha  ventas_totales  ventas_dia_anterior  diferencia_porcentual
64  2023-05-06          985.00                83.00                1086.75
148 2023-07-29          485.25                46.75                 937.97
3   2023-03-06          723.00                75.00                 864.00
--------
Top 3 días con MAYOR caída porcentual:
         fecha  ventas_totales  ventas_dia_anterior  diferencia_porcentual
174 2023-08-24           39.25               590.15                 -93.35
54  2023-04-26          119.00               750.00                 -84.13
27  2023-03-30           79.00               452.00                 -82.52


### Ejercicio 5: Análisis de Fin de Semana vs Entre Semana
Compara el comportamiento de compra entre fin de semana (sábado y domingo) y entre semana:

Para cada segmento calcula:
- Número de órdenes únicas
- Ticket medio (gasto medio por orden)
- Productos por orden (media)
- Familia de productos más vendida
- Porcentaje de ventas de cada familia

Presenta los resultados en un formato que permita comparar fácilmente ambos segmentos.

In [8]:
# Ejercicio 5: Análisis Fin de Semana vs Entre Semana

# Asegurar que fecha es datetime
data['fecha'] = pd.to_datetime(data['fecha'])

# Crear columna día de la semana (0=lunes, 6=domingo)
data['is_weekend'] = data['fecha'].dt.day_name().isin(["Sunday", "Saturday"])

compras = data.groupby("is_weekend", as_index = False).agg({
    'order_id': 'nunique',  # Número de órdenes únicas
    'price': 'sum',        # Ventas totales
    'product_id': 'nunique',  # Número de productos vendidos
})

compras.columns = ['is_weekend', 'num_ordenes', 'gasto_total', 'num_productos_vendidos']
compras["gasto_medio"]= compras["gasto_total"] / compras["num_ordenes"]
compras["productos_por_orden"] = compras["num_productos_vendidos"] / compras["num_ordenes"]



# Obtenemos la familia de productos favorita 
compras_por_familia = data.groupby(["is_weekend", "product_family"], as_index = False)["quantity"].sum()
familia_favorita = compras_por_familia.iloc[compras_por_familia.groupby("is_weekend")["quantity"].idxmax()]
familia_favorita.columns = ["is_weekend", "familia_favorita", "cantidad_familia_favorita"]

compras = compras.merge(familia_favorita, on = "is_weekend")
compras["porcentaje_familia_favorita"] = 100 * compras["cantidad_familia_favorita"] / compras["num_productos_vendidos"]
compras.drop(columns = ["cantidad_familia_favorita", "num_productos_vendidos"], inplace = True)
print(compras)


   is_weekend  num_ordenes  gasto_total  gasto_medio  productos_por_orden  \
0       False         1129     53863.47    47.709008             0.234721   
1        True          528     26189.40    49.601136             0.409091   

  familia_favorita  porcentaje_familia_favorita  
0           Collar                   289.433962  
1           Collar                   186.111111  


### Ejercicio 6: Cohort Analysis Simplificado
Realiza un análisis de cohortes básico:

1. Para cada cliente, identifica la fecha de su primera compra (fecha de adquisición)
2. Agrupa a los clientes por semana de adquisición
3. Para cada cohorte, calcula:
   - Número de clientes en la cohorte
   - Gasto total de la cohorte
   - Gasto medio por cliente
   - Número medio de órdenes por cliente
   
Ordena las cohortes de más antigua a más reciente.

In [9]:
# Ejercicio 6: Cohort Analysis Simplificado

# Asegurar que fecha es datetime
data['fecha'] = pd.to_datetime(data['fecha'])
data["fecha_week_start"] = data['fecha'].dt.to_period('W').dt.start_time


# Calcular agregaciones por cliente
compras = data.groupby("order_customer_id", as_index = False).agg({
    'fecha_week_start': 'min',
    'price': 'sum',
    'order_id': 'nunique' 
})
compras.columns = ['order_customer_id', 'fecha_primera_compra', 'gasto_total', 'num_ordenes']

print(compras.sort_values("fecha_primera_compra"))

# Calcular agregaciones por cohorte
compras = compras.groupby("fecha_primera_compra", as_index = False).agg({
    'order_customer_id': 'nunique',
    'gasto_total': ['sum', 'mean'],
    'num_ordenes': 'mean'
})
compras.columns = ['fecha_primera_compra', 'num_clientes', 'gasto_total', 'gasto_medio', 'ordenes_medias']

print(compras.sort_values("fecha_primera_compra"))



      order_customer_id fecha_primera_compra  gasto_total  num_ordenes
143          6618401292           2023-02-27         89.0            1
261         55872635468           2023-02-27        120.0            3
349        206959821388           2023-02-27         89.0            1
400        241611129420           2023-02-27         75.0            1
284        105953837644           2023-02-27         83.0            1
...                 ...                  ...          ...          ...
177          6893212300           2023-10-02         29.0            1
816        616726282923           2023-10-02         19.0            1
101          5987654092           2023-10-02         29.0            1
1187       672710935172           2023-10-02         54.0            1
1286       713419604612           2023-10-02         58.0            1

[1314 rows x 4 columns]
   fecha_primera_compra  num_clientes  gasto_total  gasto_medio  \
0            2023-02-27            13       924.00    71


### Ejercicio 7: Elasticidad de Cantidad
Analiza cómo varía la cantidad comprada según el precio unitario:

1. Crea rangos de precio unitario (bins): 0-20, 20-40, 40-60, 60+
2. Para cada rango y cada familia de productos, calcula:
   - Cantidad media comprada
   - Número de transacciones
3. Crea una tabla pivote que muestre la cantidad media comprada con:
   - Filas: familias de productos
   - Columnas: rangos de precio
4. ¿Qué familia de productos es menos sensible al precio (mantiene cantidades altas incluso a precios altos)?

In [10]:
# Ejercicio 7: Elasticidad de Cantidad

# Crear rangos de precio unitario
data['rango_precio'] = pd.cut(data['unit_price'], 
    bins=[0, 20, 40, 60, float('inf')],
    labels=['0-20', '20-40', '40-60', '60+'],
    right=False
)

# Calcular métricas por rango de precio y familia
elasticidad = data.groupby(['product_family', 'rango_precio']).agg({
    'product_id': 'nunique',  # Número de productos
    'quantity': 'sum',  # Cantidad total comprada
    'order_id': 'nunique',  # Número de transacciones
}).reset_index()

elasticidad.columns = ['familia', 'rango_precio', 'numero_productos', 'cantidad_total', 'num_transacciones']
elasticidad["cantidad_media"] = elasticidad["cantidad_total"] / elasticidad["numero_productos"]
# Crear tabla pivote: cantidad media con familias en filas y rangos en columnas
tabla_pivote = elasticidad.pivot_table(
    index='familia',
    columns='rango_precio',
    values='cantidad_media',
    fill_value=0
).round(2)

print("=== Tabla Pivote: Cantidad Media Comprada ===")
print("Filas: Familias de productos | Columnas: Rangos de precio\n")
print(tabla_pivote)
print("\n")


=== Tabla Pivote: Cantidad Media Comprada ===
Filas: Familias de productos | Columnas: Rangos de precio

rango_precio   0-20  20-40  40-60  60+
familia                               
Anillo         6.12   3.37   0.00  0.0
Collar        13.08  18.62   4.31  4.0
Pendientes     7.10   9.96   3.00  1.0
Pulsera        7.76   7.32   7.00  0.0
Tobillera     12.67  25.38   0.00  0.0




/tmp/ipykernel_865681/2020987983.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  elasticidad = data.groupby(['product_family', 'rango_precio']).agg({
/tmp/ipykernel_865681/2020987983.py:20: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tabla_pivote = elasticidad.pivot_table(


### Ejercicio 8: Análisis de Margen de Contribución
Simula el coste del producto (como mucho 40% del `unit_price`). Calcula para cada familia de productos:

1. Margen unitario promedio (unit_price - coste)
2. Margen total (suma de todos los márgenes de las ventas)
3. Contribución al margen total del negocio (porcentaje)

Ordena por contribución al margen descendente. ¿Cuál es la familia más rentable?

In [ ]:
# Ejercicio 8: Análisis de Margen de Contribución

# Simular coste del producto (40% del unit_price)
data['coste'] = np.random.uniform(0.1, data['unit_price'] * 0.4)

# Calcular margen unitario
data['margen_unitario'] = data['unit_price'] - data['coste']

# Calcular margen por transacción (margen_unitario * quantity)
data['margen_transaccion'] = data['margen_unitario'] * data['quantity']

# Análisis por familia de productos
margen_familia = data.groupby('product_family').agg({
    'product_id': 'nunique',  # Número de productos
    'margen_unitario': 'sum',  # Suma margen unitario 
    'margen_transaccion': 'sum',  # Margen total
}).reset_index()

margen_familia.columns = ['familia','numero_productos', 'margen_unitario_sum', 'margen_total']
margen_familia['margen_unitario_promedio'] = (
    margen_familia['margen_unitario_sum'] / margen_familia['numero_productos']
)

# Calcular contribución al margen total del negocio
margen_total_negocio = margen_familia['margen_total'].sum()
margen_familia['contribucion_porcentaje'] = (
    margen_familia['margen_total'] / margen_total_negocio * 100
).round(2)

# Ordenar por contribución descendente
margen_familia = margen_familia.sort_values('contribucion_porcentaje', ascending=False)

print("=== Análisis de Margen de Contribución por Familia ===\n")
print(margen_familia)
print(f"\n{'='*60}")

=== Análisis de Margen de Contribución por Familia ===

      familia  margen_unitario_promedio  margen_total  contribucion_porcentaje
1      Collar                 22.614425  26400.824733                    41.13
2  Pendientes                 20.172442  13538.783472                    21.09
3     Pulsera                 19.796916   9473.357783                    14.76
0      Anillo                 18.381840   9344.894488                    14.56
4   Tobillera                 19.520266   5425.485416                     8.45



### Ejercicio 9: Segmentación RFM (Recency, Frequency, Monetary)
Crea una segmentación RFM de clientes:

1. **Recency**: Días desde la última compra (usa la fecha más reciente del dataset como referencia)
2. **Frequency**: Número de órdenes del cliente
3. **Monetary**: Gasto total del cliente

Para cada métrica:
- Asigna un score de 1 a 4 usando cuartiles (1=peor, 4=mejor)
- Para Recency, invierte la lógica (menor recency = mejor = score más alto)

Crea una columna `RFM_Score` concatenando los tres scores (ej: "444" = cliente excelente).

Finalmente, clasifica a los clientes en segmentos de tu ínteres.

¿Cuántos clientes hay en cada segmento?

In [22]:
# Ejercicio 9: Segmentación RFM (Recency, Frequency, Monetary)

# Asegurar que fecha es datetime
data['fecha'] = pd.to_datetime(data['fecha'])

# Fecha de referencia (fecha más reciente del dataset)
fecha_referencia = data['fecha'].max()

# Calcular métricas RFM por cliente
rfm = data.groupby('order_customer_id', as_index = False).agg({
    'fecha': lambda x: (fecha_referencia - x.max()).days,  # Recency: días desde última compra
    'order_id': 'nunique',  # Frequency: número de órdenes
    'price': 'sum'  # Monetary: gasto total
})

rfm.columns = ['order_customer_id', 'recency', 'frequency', 'monetary']

# Asignar scores usando cuartiles (1=peor, 4=mejor)
# Para Recency: menor es mejor (invertir lógica)
rfm['R_score'] = pd.qcut(rfm['recency'], q=4, labels=[4, 3, 2, 1])
rfm['F_score'] = pd.qcut(rfm['frequency'].rank(method='first'), q=4, labels=[1, 2, 3, 4])
rfm['M_score'] = pd.qcut(rfm['monetary'].rank(method='first'), q=4, labels=[1, 2, 3, 4])

# # Convertir scores a string y concatenar
rfm['RFM_Score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)
rfm

# Clasificar en segmentos
def clasificar_segmento(row):
    r, f, m = int(row['R_score']), int(row['F_score']), int(row['M_score'])
    
    # Clientes excelentes (Champions)
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    # Clientes leales (Loyal)
    elif f >= 3 and m >= 3:
        return 'Loyal Customers'
    # Clientes potenciales (Potential)
    elif r >= 3 and f <= 2 and m <= 2:
        return 'Potential Loyalists'
    # Nuevos clientes (New)
    elif r >= 4 and f <= 2:
        return 'New Customers'
    # En riesgo (At Risk)
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'
    # Perdidos (Lost)
    elif r <= 2 and f <= 2:
        return 'Lost'
    else:
        return 'Others'

rfm['segmento'] = rfm.apply(clasificar_segmento, axis=1)

print("=== Segmentación RFM de Clientes ===\n")

# Contar clientes por segmento
rfm.groupby('segmento')["order_customer_id"].count()

=== Segmentación RFM de Clientes ===



segmento
Champions               94
Lost                   530
Loyal Customers        302
New Customers           22
Others                 287
Potential Loyalists     79
Name: order_customer_id, dtype: int64

### Ejercicio 10: Análisis de Pareto (80/20)
Realiza un análisis de Pareto para identificar:

1. **A nivel de clientes**: ¿Qué porcentaje de clientes genera el 80% de los ingresos?
2. **A nivel de productos**: ¿Qué porcentaje de familias de productos genera el 80% de los ingresos?
3. **A nivel de órdenes**: ¿Qué porcentaje de órdenes representa el 80% de los ingresos?

Para cada análisis:
- Ordena por ingresos descendente
- Calcula el porcentaje acumulado de ingresos
- Identifica el punto donde se alcanza el 80%
- Calcula qué porcentaje del total representa ese punto

In [21]:
# Ejercicio 10: Análisis de Pareto (80/20)

print("=== Análisis de Pareto (80/20) ===\n")

# 1. Análisis a nivel de CLIENTES
print("1. ANÁLISIS A NIVEL DE CLIENTES")
print("-" * 60)

# Agrupar por cliente y calcular ingresos totales
clientes_ingresos = data.groupby('order_customer_id', as_index=False)['price'].sum()
clientes_ingresos.columns = ['order_customer_id', 'ingresos_totales']

# Ordenar por ingresos descendente
clientes_ingresos = clientes_ingresos.sort_values('ingresos_totales', ascending=False)

# Calcular porcentaje acumulado de ingresos
clientes_ingresos['ingresos_acumulados'] = clientes_ingresos['ingresos_totales'].cumsum()
total_ingresos = clientes_ingresos['ingresos_totales'].sum()
clientes_ingresos['porcentaje_acumulado'] = (clientes_ingresos['ingresos_acumulados'] / total_ingresos * 100).round(2)

# Identificar el punto donde se alcanza el 80%
clientes_80 = clientes_ingresos[clientes_ingresos['porcentaje_acumulado'] <= 80]
num_clientes_80 = len(clientes_80)
total_clientes = len(clientes_ingresos)
porcentaje_clientes_80 = (num_clientes_80 / total_clientes * 100)

print(f"Total de clientes: {total_clientes}")
print(f"Clientes que generan el 80% de los ingresos: {num_clientes_80}")
print(f"Porcentaje de clientes: {porcentaje_clientes_80}%")
print(f"Ingresos generados por estos clientes: €{clientes_80['ingresos_totales'].sum():.2f}")
print()

# 2. Análisis a nivel de FAMILIAS DE PRODUCTOS
print("2. ANÁLISIS A NIVEL DE FAMILIAS DE PRODUCTOS")
print("-" * 60)

# Agrupar por familia y calcular ingresos totales
familias_ingresos = data.groupby('product_family', as_index=False)['price'].sum()
familias_ingresos.columns = ['product_family', 'ingresos_totales']

# Ordenar por ingresos descendente
familias_ingresos = familias_ingresos.sort_values('ingresos_totales', ascending=False)

# Calcular porcentaje acumulado de ingresos
familias_ingresos['ingresos_acumulados'] = familias_ingresos['ingresos_totales'].cumsum()
familias_ingresos['porcentaje_acumulado'] = (familias_ingresos['ingresos_acumulados'] / total_ingresos * 100).round(2)

# Identificar el punto donde se alcanza el 80%
familias_80 = familias_ingresos[familias_ingresos['porcentaje_acumulado'] <= 80]
num_familias_80 = len(familias_80)
total_familias = len(familias_ingresos)
porcentaje_familias_80 = (num_familias_80 / total_familias * 100)

print(f"Total de familias de productos: {total_familias}")
print(f"Familias que generan el 80% de los ingresos: {num_familias_80}")
print(f"Porcentaje de familias: {porcentaje_familias_80}%")
print(f"Familias: {familias_80['product_family'].tolist()}")
print()

# 3. Análisis a nivel de ÓRDENES
print("3. ANÁLISIS A NIVEL DE ÓRDENES")
print("-" * 60)

# Agrupar por orden y calcular ingresos totales
ordenes_ingresos = data.groupby('order_id', as_index=False)['price'].sum()
ordenes_ingresos.columns = ['order_id', 'ingresos_totales']

# Ordenar por ingresos descendente
ordenes_ingresos = ordenes_ingresos.sort_values('ingresos_totales', ascending=False)

# Calcular porcentaje acumulado de ingresos
ordenes_ingresos['ingresos_acumulados'] = ordenes_ingresos['ingresos_totales'].cumsum()
ordenes_ingresos['porcentaje_acumulado'] = (ordenes_ingresos['ingresos_acumulados'] / total_ingresos * 100).round(2)

# Identificar el punto donde se alcanza el 80%
ordenes_80 = ordenes_ingresos[ordenes_ingresos['porcentaje_acumulado'] <= 80]
num_ordenes_80 = len(ordenes_80)
total_ordenes = len(ordenes_ingresos)
porcentaje_ordenes_80 = (num_ordenes_80 / total_ordenes * 100)

print(f"Total de órdenes: {total_ordenes}")
print(f"Órdenes que representan el 80% de los ingresos: {num_ordenes_80}")
print(f"Porcentaje de órdenes: {porcentaje_ordenes_80}%")
print(f"Ingresos generados por estas órdenes: €{ordenes_80['ingresos_totales'].sum():.2f}")


=== Análisis de Pareto (80/20) ===

1. ANÁLISIS A NIVEL DE CLIENTES
------------------------------------------------------------
Total de clientes: 1314
Clientes que generan el 80% de los ingresos: 731
Porcentaje de clientes: 55.63165905631659%
Ingresos generados por estos clientes: €64044.62

2. ANÁLISIS A NIVEL DE FAMILIAS DE PRODUCTOS
------------------------------------------------------------
Total de familias de productos: 5
Familias que generan el 80% de los ingresos: 3
Porcentaje de familias: 60.0%
Familias: ['Collar', 'Pendientes', 'Pulsera']

3. ANÁLISIS A NIVEL DE ÓRDENES
------------------------------------------------------------
Total de órdenes: 1657
Órdenes que representan el 80% de los ingresos: 1012
Porcentaje de órdenes: 61.07423053711527%
Ingresos generados por estas órdenes: €64018.27


### Ejercicio 11: Análisis de Días Anómalos
Identifica días con comportamiento de ventas anómalo:

1. Calcula las ventas diarias totales
2. Calcula la media y desviación estándar de las ventas diarias
3. Identifica días "anómalos":
   - Días con ventas > media + 1.5*desviación_estándar (picos)
   - Días con ventas < media - 1.5*desviación_estándar (caídas)

4. Para los días anómalos, analiza:
   - ¿Qué día de la semana son?
   - ¿Qué porcentaje de ventas aportaron cada familia en esos días?

In [20]:
# Ejercicio 11: Análisis de Días Anómalos

# Asegurar que fecha es datetime
data['fecha'] = pd.to_datetime(data['fecha'])

# 1. Calcular ventas diarias totales
ventas_diarias = data.groupby('fecha', as_index=False)['price'].sum()
ventas_diarias.columns = ['fecha', 'ventas_totales']

# 2. Calcular media y desviación estándar
media_ventas = ventas_diarias['ventas_totales'].mean()
std_ventas = ventas_diarias['ventas_totales'].std()

print("=== Análisis de Días Anómalos ===\n")
print(f"Media de ventas diarias: €{media_ventas:.2f}")
print(f"Desviación estándar: €{std_ventas:.2f}")
print()

# 3. Identificar días anómalos
# Umbrales
umbral_superior = media_ventas + 1.5 * std_ventas
umbral_inferior = media_ventas - 1.5 * std_ventas

print(f"Umbral superior (picos): €{umbral_superior:.2f}")
print(f"Umbral inferior (caídas): €{umbral_inferior:.2f}")
print()

# Clasificar días
ventas_diarias['tipo_dia'] = 'Normal'
ventas_diarias.loc[ventas_diarias['ventas_totales'] > umbral_superior, 'tipo_dia'] = 'Pico'
ventas_diarias.loc[ventas_diarias['ventas_totales'] < umbral_inferior, 'tipo_dia'] = 'Caída'

# Filtrar días anómalos
dias_anomalos = ventas_diarias[ventas_diarias['tipo_dia'] != 'Normal'].copy()

print(f"Total de días analizados: {len(ventas_diarias)}")
print(f"Días con picos de ventas: {len(dias_anomalos[dias_anomalos['tipo_dia'] == 'Pico'])}")
print(f"Días con caídas de ventas: {len(dias_anomalos[dias_anomalos['tipo_dia'] == 'Caída'])}")
print()

# 4. Análisis de días anómalos

if len(dias_anomalos) > 0:
    # Añadir día de la semana
    dias_anomalos['dia_semana'] = dias_anomalos['fecha'].dt.day_name()
    
    # Análisis por día de la semana
    print("=" * 80)
    print("DISTRIBUCIÓN POR DÍA DE LA SEMANA")
    print("=" * 80)
    distribucion_dias = dias_anomalos.groupby(['tipo_dia', 'dia_semana']).size().reset_index(name='count')
    print(distribucion_dias)
    print()
    
    # Análisis de familias de productos para días anómalos
    print("=" * 80)
    print("COMPOSICIÓN DE VENTAS POR FAMILIA EN DÍAS ANÓMALOS")
    print("=" * 80)
    
    for fecha_anomala in dias_anomalos['fecha'].values:
        # Filtrar datos del día anómalo
        ventas_dia = data[data['fecha'] == fecha_anomala]
        tipo_dia = dias_anomalos[dias_anomalos['fecha'] == fecha_anomala]['tipo_dia'].values[0]
        dia_semana = pd.Timestamp(fecha_anomala).day_name()
        total_dia = ventas_dia['price'].sum()
        
        # Calcular porcentaje por familia
        ventas_por_familia = ventas_dia.groupby('product_family')['price'].sum().reset_index()
        ventas_por_familia['porcentaje'] = (ventas_por_familia['price'] / total_dia * 100).round(2)
        ventas_por_familia = ventas_por_familia.sort_values('porcentaje', ascending=False)
        
        print(f"\n{tipo_dia.upper()}: {pd.Timestamp(fecha_anomala).strftime('%Y-%m-%d')} ({dia_semana})")
        print(f"Total ventas: €{total_dia:.2f}")
        print("-" * 60)
        for _, row in ventas_por_familia.iterrows():
            print(f"  {row['product_family']:15s} €{row['price']:7.2f} ({row['porcentaje']:5.2f}%)")
        print()
    
else:
    print("No se encontraron días anómalos con los criterios establecidos.")

=== Análisis de Días Anómalos ===

Media de ventas diarias: €375.84
Desviación estándar: €246.57

Umbral superior (picos): €745.69
Umbral inferior (caídas): €5.98

Total de días analizados: 213
Días con picos de ventas: 15
Días con caídas de ventas: 0

DISTRIBUCIÓN POR DÍA DE LA SEMANA
  tipo_dia dia_semana  count
0     Pico     Friday      1
1     Pico     Monday      3
2     Pico   Saturday      7
3     Pico     Sunday      2
4     Pico    Tuesday      2

COMPOSICIÓN DE VENTAS POR FAMILIA EN DÍAS ANÓMALOS

PICO: 2023-03-11 (Saturday)
Total ventas: €1034.00
------------------------------------------------------------
  Collar          € 487.00 (47.10%)
  Pendientes      € 295.00 (28.53%)
  Anillo          € 164.00 (15.86%)
  Pulsera         €  88.00 ( 8.51%)


PICO: 2023-03-12 (Sunday)
Total ventas: €944.00
------------------------------------------------------------
  Collar          € 318.00 (33.69%)
  Pendientes      € 262.00 (27.75%)
  Pulsera         € 235.00 (24.89%)
  Anillo   

### Ejercicio 12: Predicción de Próxima Compra
Para cada cliente que haya comprado más de una vez:

1. Calcula el tiempo medio entre compras (en días)
2. Calcula la desviación estándar del tiempo entre compras
3. Identifica la fecha de la última compra
4. Predice cuándo hará su próxima compra: última_compra + tiempo_medio
5. Calcula un "intervalo de confianza" simple: próxima_compra ± desviación_estándar

Clasifica a los clientes en:
- "Debería comprar pronto": próxima compra predicha ya pasó
- "Compra inminente": próxima compra en los próximos 7 días
- "No urgente": próxima compra en más de 7 días

In [ ]:
# Ejercicio 12: Predicción de Próxima Compra

# Asegurar que fecha es datetime
data['fecha'] = pd.to_datetime(data['fecha'])

# Fecha de referencia (fecha más reciente del dataset)
fecha_referencia = data['fecha'].max()

print("=== Predicción de Próxima Compra ===\n")
print(f"Fecha de referencia (hoy): {fecha_referencia.strftime('%Y-%m-%d')}\n")

# Obtener todas las fechas de compra por cliente
fechas_por_cliente = data.groupby('order_customer_id')['fecha'].apply(lambda x: sorted(x.unique())).reset_index()
fechas_por_cliente.columns = ['order_customer_id', 'fechas_compras']

# Filtrar clientes que han comprado más de una vez
fechas_por_cliente['num_compras'] = fechas_por_cliente['fechas_compras'].apply(len)
clientes_recurrentes = fechas_por_cliente[fechas_por_cliente['num_compras'] > 1]

print(f"Total de clientes: {len(fechas_por_cliente)}")
print(f"Clientes recurrentes (más de 1 compra): {len(clientes_recurrentes)}")
print()

# Calcular métricas para cada cliente recurrente
def calcular_metricas_cliente(fechas):
    fechas = sorted(fechas)
    
    # Calcular diferencias entre compras consecutivas (en días)
    diferencias = [(fechas[i+1] - fechas[i]).days for i in range(len(fechas)-1)]
    
    # Tiempo medio entre compras
    tiempo_medio = np.mean(diferencias)
    
    # Desviación estándar
    std_tiempo = np.std(diferencias) if len(diferencias) > 1 else 0
    
    # Última compra
    ultima_compra = fechas[-1]
    
    # Predicción de próxima compra
    proxima_compra = ultima_compra + pd.Timedelta(days=tiempo_medio)
    
    # Intervalo de confianza
    intervalo_inferior = proxima_compra - pd.Timedelta(days=std_tiempo)
    intervalo_superior = proxima_compra + pd.Timedelta(days=std_tiempo)
    
    return {
        'tiempo_medio_dias': tiempo_medio,
        'std_tiempo_dias': std_tiempo,
        'ultima_compra': ultima_compra,
        'proxima_compra_predicha': proxima_compra,
        'intervalo_inferior': intervalo_inferior,
        'intervalo_superior': intervalo_superior
    }

# Aplicar cálculos a cada cliente
metricas = clientes_recurrentes['fechas_compras'].apply(calcular_metricas_cliente)
metricas_df = pd.DataFrame(metricas.tolist())

# Combinar con los IDs de cliente
predicciones = pd.concat([
    clientes_recurrentes[['order_customer_id', 'num_compras']].reset_index(drop=True),
    metricas_df
], axis=1)

# Calcular días hasta la próxima compra predicha
predicciones['dias_hasta_proxima'] = (predicciones['proxima_compra_predicha'] - fecha_referencia).dt.days

# Clasificar clientes
def clasificar_cliente(dias):
    if dias < 0:
        return 'Debería comprar pronto'
    elif dias <= 7:
        return 'Compra inminente'
    else:
        return 'No urgente'

predicciones['clasificacion'] = predicciones['dias_hasta_proxima'].apply(clasificar_cliente)

# Mostrar resultados
print("=" * 80)
print("RESUMEN DE CLASIFICACIÓN")
print("=" * 80)
clasificacion_resumen = predicciones['clasificacion'].value_counts().sort_index()
print(clasificacion_resumen)
print()
predicciones

=== Predicción de Próxima Compra ===

Fecha de referencia (hoy): 2023-10-03

Total de clientes: 1314
Clientes recurrentes (más de 1 compra): 247

RESUMEN DE CLASIFICACIÓN
clasificacion
Compra inminente            7
Debería comprar pronto    170
No urgente                 70
Name: count, dtype: int64



,order_customer_id,num_compras,tiempo_medio_dias,std_tiempo_dias,ultima_compra,proxima_compra_predicha,intervalo_inferior,intervalo_superior,dias_hasta_proxima,clasificacion
0,4438242304,3,46.5,41.5,2023-09-07,2023-10-23 12:00:00,2023-09-12,2023-12-04,20,No urgente
1,4438581888,2,141.0,0.0,2023-09-25,2024-02-13 00:00:00,2024-02-13,2024-02-13,133,No urgente
2,4438723840,2,79.0,0.0,2023-08-22,2023-11-09 00:00:00,2023-11-09,2023-11-09,37,No urgente
3,4438983936,2,41.0,0.0,2023-07-16,2023-08-26 00:00:00,2023-08-26,2023-08-26,-38,Debería comprar pronto
4,4440001280,2,24.0,0.0,2023-05-30,2023-06-23 00:00:00,2023-06-23,2023-06-23,-102,Debería comprar pronto
...,...,...,...,...,...,...,...,...,...,...
242,699735622276,2,16.0,0.0,2023-09-23,2023-10-09 00:00:00,2023-10-09,2023-10-09,6,Compra inminente
243,713373139588,2,17.0,0.0,2023-09-14,2023-10-01 00:00:00,2023-10-01,2023-10-01,-2,Debería comprar pronto
244,713721266820,2,10.0,0.0,2023-09-07,2023-09-17 00:00:00,2023-09-17,2023-09-17,-16,Debería comprar pronto
245,716591776388,2,7.0,0.0,2023-09-07,2023-09-14 00:00:00,2023-09-14,2023-09-14,-19,Debería comprar pronto


### Ejercicio 13: Reporte Ejecutivo Automatizado
Crea una función que genere un reporte ejecutivo automático:

La función debe:
1. Recibir el DataFrame y un rango de fechas
2. Filtrar los datos por ese rango
3. Calcular y retornar un diccionario con:
   - Resumen de ventas (total, media diaria, tendencia)
   - Top 5 clientes
   - Top 3 productos

Prueba la función con diferentes rangos de fechas.

In [24]:
# Ejercicio 13: Reporte Ejecutivo Automatizado

def generar_reporte_ejecutivo(df, fecha_inicio, fecha_fin):
    """
    Genera un reporte ejecutivo automático para un rango de fechas.
    
    Parámetros:
    - df: DataFrame con los datos de ventas
    - fecha_inicio: fecha inicial del período (string 'YYYY-MM-DD')
    - fecha_fin: fecha final del período (string 'YYYY-MM-DD')
    
    Retorna:
    - diccionario con el reporte completo
    """
    # Asegurar que fecha es datetime
    df['fecha'] = pd.to_datetime(df['fecha'])
    
    # Convertir fechas de entrada a datetime
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_fin = pd.to_datetime(fecha_fin)
    
    # Filtrar datos por rango de fechas
    df_filtrado = df[(df['fecha'] >= fecha_inicio) & (df['fecha'] <= fecha_fin)].copy()
    
    # ===============================================
    # 1. RESUMEN DE VENTAS
    # ===============================================
    ventas_totales = df_filtrado['price'].sum()
    num_dias = (fecha_fin - fecha_inicio).days + 1
    media_diaria = ventas_totales / num_dias
    
    # Calcular tendencia (ventas primera mitad vs segunda mitad)
    fecha_media = fecha_inicio + (fecha_fin - fecha_inicio) / 2
    ventas_primera_mitad = df_filtrado[df_filtrado['fecha'] < fecha_media]['price'].sum()
    ventas_segunda_mitad = df_filtrado[df_filtrado['fecha'] >= fecha_media]['price'].sum()
    
    if ventas_primera_mitad > 0:
        tendencia_porcentaje = ((ventas_segunda_mitad - ventas_primera_mitad) / ventas_primera_mitad) * 100
        tendencia = 'Creciente' if tendencia_porcentaje > 5 else ('Decreciente' if tendencia_porcentaje < -5 else 'Estable')
    else:
        tendencia_porcentaje = 0
        tendencia = 'Sin datos suficientes'
    
    resumen_ventas = {
        'ventas_totales': round(ventas_totales, 2),
        'media_diaria': round(media_diaria, 2),
        'num_dias': num_dias,
        'num_ordenes': df_filtrado['order_id'].nunique(),
        'ticket_medio': round(ventas_totales / df_filtrado['order_id'].nunique(), 2) if df_filtrado['order_id'].nunique() > 0 else 0,
        'tendencia': tendencia,
        'tendencia_porcentaje': round(tendencia_porcentaje, 2)
    }
    
    # ===============================================
    # 2. TOP 5 CLIENTES
    # ===============================================
    top_clientes = df_filtrado.groupby('order_customer_id', as_index=False).agg({
        'price': 'sum',
        'order_id': 'nunique'
    }).sort_values('price', ascending=False).head(5)
    top_clientes.columns = ['customer_id', 'gasto_total', 'num_ordenes']
    top_clientes['ticket_medio'] = (top_clientes['gasto_total'] / top_clientes['num_ordenes']).round(2)
    
    # ===============================================
    # 3. TOP 3 FAMILIAS DE PRODUCTOS
    # ===============================================
    top_productos = df_filtrado.groupby('product_family', as_index=False).agg({
        'price': 'sum',
        'quantity': 'sum'
    }).sort_values('price', ascending=False).head(3)
    top_productos.columns = ['familia', 'ingresos', 'unidades_vendidas']
    top_productos['porcentaje_ingresos'] = (top_productos['ingresos'] / ventas_totales * 100).round(2)
    
    
    # ===============================================
    # 6. CONSTRUIR REPORTE FINAL
    # ===============================================
    reporte = {
        'periodo': {
            'fecha_inicio': fecha_inicio.strftime('%Y-%m-%d'),
            'fecha_fin': fecha_fin.strftime('%Y-%m-%d'),
            'dias_analizados': num_dias
        },
        'resumen_ventas': resumen_ventas,
        'top_clientes': top_clientes.to_dict('records'),
        'top_productos': top_productos.to_dict('records'),
    }
    
    return reporte


def imprimir_reporte(reporte):
    """
    Imprime el reporte de manera formateada y legible.
    """
    if 'error' in reporte:
        print(f"\n❌ ERROR: {reporte['error']}")
        return
    
    print("\n" + "="*100)
    print("📊 REPORTE EJECUTIVO DE VENTAS")
    print("="*100)
    
    # Período
    print(f"\n📅 PERÍODO ANALIZADO")
    print(f"   Desde: {reporte['periodo']['fecha_inicio']} | Hasta: {reporte['periodo']['fecha_fin']}")
    print(f"   Días analizados: {reporte['periodo']['dias_analizados']}")
    
    # Resumen de ventas
    print(f"\n💰 RESUMEN DE VENTAS")
    print(reporte)
    # Top clientes
    
print("🚀 GENERANDO REPORTES EJECUTIVOS...\n")

# Reporte 1: Marzo 2023
print("\n" + "#"*100)
print("# REPORTE 1: MARZO 2023")
print("#"*100)
reporte1 = generar_reporte_ejecutivo(data, '2023-03-01', '2023-03-31')
imprimir_reporte(reporte1)

🚀 GENERANDO REPORTES EJECUTIVOS...


####################################################################################################
# REPORTE 1: MARZO 2023
####################################################################################################

📊 REPORTE EJECUTIVO DE VENTAS

📅 PERÍODO ANALIZADO
   Desde: 2023-03-01 | Hasta: 2023-03-31
   Días analizados: 31

💰 RESUMEN DE VENTAS
{'periodo': {'fecha_inicio': '2023-03-01', 'fecha_fin': '2023-03-31', 'dias_analizados': 31}, 'resumen_ventas': {'ventas_totales': np.float64(14410.5), 'media_diaria': np.float64(464.85), 'num_dias': 31, 'num_ordenes': 224, 'ticket_medio': np.float64(64.33), 'tendencia': 'Creciente', 'tendencia_porcentaje': np.float64(10.83)}, 'top_clientes': [{'customer_id': 244904182348, 'gasto_total': 243.0, 'num_ordenes': 2, 'ticket_medio': 121.5}, {'customer_id': 215597728332, 'gasto_total': 204.5, 'num_ordenes': 2, 'ticket_medio': 102.25}, {'customer_id': 257576976972, 'gasto_total': 166.0, 'num_ordenes'